In [1]:
!pip3 install transformers
!pip3 install datasets
!pip3 install rouge_score
!pip install 'gem-metrics @ git+https://github.com/GEM-benchmark/GEM-metrics.git' bert_score
import transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 9.5 MB/s 
     |████████████████████████████████| 163 kB 55.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 7.2 MB/s 
     |████████████████████████████████| 212 kB 57.4 MB/s 
     |████████████████████████████████| 115 kB 58.7 MB/s 
     |████████████████████████████████| 127 kB 56.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24955 sha256=46aab59dcaf80fc0bbb16a0bac20ab21532cfee860116f

In [1]:
from huggingface_hub import notebook_login
notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Sandipan1994/t5-small-finetuned-eli5")
model = AutoModelForSeq2SeqLM.from_pretrained("Sandipan1994/t5-small-finetuned-eli5")

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("eli5", split="test_eli5")
metric = load_metric("rouge")

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Downloading:   0%|          | 0.00/3.50k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/21.1M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/53.0M [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/330M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/18.7M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/36.2M [00:00<?, ?B/s]

Dataset eli5 downloaded and prepared to /root/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa. Subsequent calls will reuse this data.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  after removing the cwd from sys.path.


In [4]:
qprefix = "question:"
cprefix = "context:"

def prep_data(examples):
  num_sents = 100
  examples['questions'] = [qprefix + " " + doc[0] + " " + cprefix + " " + doc[1] for doc in zip(examples["title"], examples['selftext'])]
  examples['concat_answers'] = [" ".join(p['text']) for p in examples['answers']]
  return examples

In [7]:
import random
def generate_random_numbers(limit = 2000, nums = 100):
  temps = []
  for i in range(nums):
    x = random.randint(0, limit)
    while x in temps:
       x = random.randint(0, limit)
    temps.append(x)
  return temps

temps = generate_random_numbers()

In [36]:
test_data = raw_datasets.filter(lambda e, i: i < 100 , with_indices=True)
test_data = test_data.map(prep_data, batched = True)

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
base_answers = test_data['concat_answers']
base_questions = test_data['questions']

In [151]:
from transformers import pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

##### vars 
max_length = 1024
min_length = 512
ngram = 2
beam = 6


model_checkpoint = "Sandipan1994/t5-small-finetuned-eli5"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint, max_length = max_length, min_length = min_length, no_repeat_ngram_size = ngram, num_beams = beam)

pipe_normal = pipeline("text2text-generation",model = model, tokenizer = tokenizer, device = 0, batch_size = 16)

In [152]:
def generate_texts(pipe, test_data = test_data):
  generated_texts = []
  for out in tqdm(pipe(KeyDataset(test_data, 'questions'))):
    generated_texts.append(out)
  test_answer_docs = []
  for ans in generated_texts:
    test_answer_docs.append(ans[0]['generated_text'])
  return test_answer_docs

test_beam = generate_texts(pipe_normal)

  0%|          | 0/7 [00:00<?, ?it/s]

In [153]:
import gem_metrics

preds_beam = gem_metrics.texts.Predictions(test_beam)
refs = gem_metrics.texts.References(base_answers)

result = gem_metrics.compute(preds_beam, refs, metrics_list=['bleu', 'rouge', 'bertscore'])

[I 221002 08:14:54 texts:54] Loading predictions for None
[I 221002 08:14:54 texts:54] Loading references for None
[I 221002 08:14:54 __init__:170] Computing BLEU for None...
[I 221002 08:14:54 __init__:170] Computing ROUGE for None...
[I 221002 08:15:07 __init__:170] Computing BERTScore for None...


In [154]:
print(result)

{'predictions_file': None, 'N': 100, 'references_file': None, 'bleu': 2.75301, 'rouge1': {'precision': 0.24098, 'recall': 0.53708, 'fmeasure': 0.27063}, 'rouge2': {'precision': 0.06119, 'recall': 0.14041, 'fmeasure': 0.06746}, 'rougeL': {'precision': 0.10115, 'recall': 0.27818, 'fmeasure': 0.1203}, 'rougeLsum': {'precision': 0.10115, 'recall': 0.27818, 'fmeasure': 0.1203}, 'bertscore': {'precision': 0.74527, 'recall': 0.78044, 'f1': 0.76175}}


In [155]:
import datetime
now = datetime.datetime.now()

with open("Result_Log.txt", "a") as fp:
  fp.write(now.strftime("%Y-%m-%d %H:%M") + "---" + "max_target_length = {}, min_target_length = {}, beam_size = {}, ngram_repeat = {}, bleu = {}, BertScore = {}, rougeL = {} ".format(max_length, min_length,beam, ngram, result['bleu'], result['bertscore'], result['rougeL']) + "\n")

In [156]:
test_beam

['I\'m not sure if you\'re a fan of music, but I don\'t think it\'s because of the fact that the music is so loud and loud that you can hear it. You\'ll hear the sound from the crowd, and you will see it in the same way as the rest of your body. This is why you get chills/goosebumps from hearing large crowds sing along to songs. _URL_0_ Is there any reason for this to happen? I have no idea what you are hearing. I think you have to be able to listen to the song. If you were to hear them, it would be better for you to feel like they\'ve been listening to them. So, i\'d say, "I think he was talking about it" or "doesnt" you would have heard it, so I would say that it is just because you hear loud noises and shits. The sound is based on the frequency of sound, which is what makes it sound like. There are many reasons why people are influenced by the noise and noise. It is also because they are more prone to noise, especially when they hear music from people who have listened to their musi